In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import time
import argparse
from timm.models.vision_transformer import VisionTransformer
import torchvision.models as models
from models import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Union, List, Dict, Any, cast

cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name, num=10):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, num)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

# -------------------- ResNet20 定义 --------------------
# (ResNet20 定义代码保持不变，此处省略以节省空间)
def _weights_init(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
    def forward(self, x):
        return self.lambd(x)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)
        self.apply(_weights_init)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride_val in strides: # Renamed variable to avoid conflict
            layers.append(block(self.in_planes, planes, stride_val)) # Use renamed variable
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        # Use adaptive average pooling for robustness to final feature map size
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet20():
    return ResNet(BasicBlock, [3, 3, 3], num_classes=100)

def ResNet32():
    return ResNet(BasicBlock, [5, 5, 5], num_classes=100)

def ResNet44():
    """ ResNet-44 model configuration based on 6n+2 formula """
    # (44-2)/6 = 7
    return ResNet(BasicBlock, [7, 7, 7], num_classes=100)

def ResNet56():
    """ ResNet-56 model configuration based on 6n+2 formula """
    # (56-2)/6 = 9
    return ResNet(BasicBlock, [9, 9, 9], num_classes=100)

def ResNet110():
    """ ResNet-110 model configuration based on 6n+2 formula """
    # (110-2)/6 = 18
    return ResNet(BasicBlock, [18, 18, 18], num_classes=100)

# -------------------- 参数设置 --------------------
parser = argparse.ArgumentParser(description='PyTorch CIFAR-100 Training')
parser.add_argument('--lr', default=0.1, type=float, help='initial learning rate')
parser.add_argument('--epochs', default=290, type=int, help='number of epochs to train')  #150 c10 resnet20 190 c100 290 vit 
parser.add_argument('--batch_size', default=128, type=int, help='batch size')
parser.add_argument('--save_path', default='./vit-t_cifar100_final_290.pth', type=str, help='path to save final model')
parser.add_argument('--data_path', default='./data', type=str, help='path to dataset')
# --- Use parse_known_args() for Jupyter compatibility ---
args, unknown = parser.parse_known_args()
# -------------------------------------------------------

# -------------------- 设备配置 --------------------
device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# -------------------- 数据准备 --------------------
print('==> Preparing data..')
cifar100_mean = (0.5071, 0.4867, 0.4408) # CIFAR-100 specific mean
cifar100_std = (0.2675, 0.2565, 0.2761)   # CIFAR-100 specific std

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cifar100_mean, cifar100_std), # <--- MODIFIED to use CIFAR-100 stats
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar100_mean, cifar100_std), # <--- MODIFIED to use CIFAR-100 stats
])

trainset = torchvision.datasets.CIFAR100( # <--- MODIFIED
    root=args.data_path, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=args.batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100( # <--- MODIFIED
    root=args.data_path, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# -------------------- 模型、损失函数、优化器、学习率调度器 --------------------
print('==> Building model..')
# net = ResNet110()
# net = ResNet56()
# vit-t
net = VisionTransformer(img_size=32, patch_size=4, num_classes=100, embed_dim=192, depth=12, num_heads=3)
# vit-s
# net = VisionTransformer(img_size=32,patch_size=4,num_classes=100, embed_dim=384, depth=12, num_heads=6)
# net = VGG('VGG11')
net = net.to(device)



# Optional DataParallel
# if device == 'cuda' and torch.cuda.device_count() > 1:
#     print(f"Let's use {torch.cuda.device_count()} GPUs!")
#     net = torch.nn.DataParallel(net)
#     cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4)
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs, eta_min=0)

# -------------------- 训练函数 --------------------
def train(epoch):
    print(f'\n--- Epoch: {epoch+1}/{args.epochs} ---')
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    start_time = time.time()
    current_lr = optimizer.param_groups[0]['lr'] # Get LR at the start of epoch

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    epoch_time = time.time() - start_time
    epoch_loss = train_loss / len(trainloader)
    epoch_acc = 100. * correct / total
    print(f'Train | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.3f}% ({correct}/{total}) | LR: {current_lr:.5f} | Time: {epoch_time:.2f}s')
    # Return values if needed elsewhere, though not strictly necessary for printing
    return epoch_loss, epoch_acc

# -------------------- 评估函数 --------------------
def evaluate(loader, set_name="Test"):
    net.eval()
    eval_loss = 0
    correct = 0
    total = 0
    start_time = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            eval_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    epoch_time = time.time() - start_time
    avg_loss = eval_loss / len(loader)
    accuracy = 100. * correct / total
    print(f'{set_name.ljust(5)} | Loss: {avg_loss:.4f} | Acc: {accuracy:.3f}% ({correct}/{total}) | Time: {epoch_time:.2f}s')
    return avg_loss, accuracy

# -------------------- 训练主循环 --------------------
print("==> Starting Training...")
training_start_time = time.time() # Use a different variable name

for epoch in range(args.epochs):
    # 1. Train for one epoch and print training metrics
    train_loss, train_acc = train(epoch)

    # 2. Evaluate on the test set after this epoch and print test metrics
    test_loss, test_acc = evaluate(testloader, "Test") # <-- Evaluate on test set

    # 3. Update the learning rate for the *next* epoch
    scheduler.step()

total_training_time = time.time() - training_start_time
print(f"\n==> Finished Training in {total_training_time:.2f} seconds ({total_training_time/3600:.2f} hours).")


# -------------------- 保存最终模型 --------------------
print(f'==> Saving final model to {args.save_path}')
save_dir = os.path.dirname(args.save_path)
if save_dir and not os.path.exists(save_dir): # Check if save_dir is not empty
    os.makedirs(save_dir)
torch.save(net.state_dict(), args.save_path)
print("Final model saved.")

# -------------------- 评估最终模型 --------------------
print("\n==> Evaluating final model (performance after {} epochs)...".format(args.epochs))
print("--- Final Training Set Evaluation ---")
# Re-evaluate on training set for final numbers (optional, could use last epoch's train results)
final_train_loss, final_train_acc = evaluate(trainloader, "Train")
print("--- Final Test Set Evaluation ---")
# Re-evaluate on test set for final numbers (optional, could use last epoch's test results)
final_test_loss, final_test_acc = evaluate(testloader, "Test")


print("\n===== Final Model Performance =====")
print(f"Training Loss: {final_train_loss:.4f}")
print(f"Training Acc:  {final_train_acc:.3f}%")
print(f"Test Loss:     {final_test_loss:.4f}")
print(f"Test Acc:      {final_test_acc:.3f}%")
print("=================================")